In [1]:
import os
import random
import torch
from d2l import torch as d2l

In [2]:
'''
数据集下载：
https://github.com/Snail1502/dataset_d2l/blob/main/wikitext-2-v1.zip
'''

'\n数据集下载：\nhttps://github.com/Snail1502/dataset_d2l/blob/main/wikitext-2-v1.zip\n'

从WikiText-2数据集中读取训练数据，并对其进行预处理，以便于BERT模型的预训练。

这段代码的核心功能是从WikiText-2数据集中读取训练数据，并将段落按句号分割成句子，同时确保每个段落至少包含两个句子。通过将所有字母转换为小写和打乱段落顺序，进一步规范化数据并增强模型的泛化能力。

In [3]:
def read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    '''
    大写字母转小写字母
    
    **处理段落**
    
    - `line.strip().lower()`：
        去除行首行尾的空白符，并将所有字母转换为小写。
    - `line.split(' . ')`：
        将段落按句号（`.`）分隔成多个句子。
    - `if len(line.split(' . ')) >= 2`：
        只保留包含至少两个句子的段落。
    '''
    paragraphs = [
        line.strip().lower().split(' . ')
        for line in lines if len(line.split(' . ')) >= 2
    ]
    '''
    **打乱段落顺序**
    
    使用 `random.shuffle` 将段落的顺序随机打乱，以增强模型的泛化能力。
    '''
    random.shuffle(paragraphs)
    return paragraphs

为预训练任务定义辅助函数

`_get_next_sentence` 和 `_get_nsp_data_from_paragraph`的目的是:
生成用于BERT模型预训练的下一句预测任务的数据。

在BERT的预训练任务中，模型需要判断两个句子是否连续出现，这被称为“下一句预测（NSP）”任务。

生成下一句预测的数据

这个函数生成一个二分类任务的训练样本，即判断两个句子是否是连续的。

In [4]:
def get_next_sentence(sentence, next_sentence, paragraphs):
    '''
    **随机选择判断标准**
    
    使用 `random.random()` 生成一个 0 到 1 之间的随机数。
    如果这个数小于 0.5，则 `is_next` 设为 `True`，
    表示 `next_sentence` 是当前 `sentence` 的下一句。
    
    否则，从段落列表 `paragraphs` 中随机选择一个句子作为 `next_sentence`，
    并将 `is_next` 设为 `False`，
    表示 `next_sentence` 不是当前 `sentence` 的下一句。

    '''
    if random.random() < 0.5:
        is_next = True
    else:
        '''
        paragraphs是三重列表的嵌套
        '''
        next_sentence = random.choice(
            random.choice(paragraphs)
        )
        is_next = False
    '''
    返回当前句子 `sentence`，
    下一句 `next_sentence`，
    以及布尔值 `is_next`，表示 `next_sentence` 是否是 `sentence` 的下一句。
    '''
    return sentence, next_sentence, is_next

In [5]:

def get_tokens_and_segments(tokens_a, tokens_b=None):
    """获取输入序列的词元及其片段索引

    Defined in :numref:`sec_bert`"""
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    # 0和1分别标记片段A和B
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>']
        segments += [1] * (len(tokens_b) + 1)
    return tokens, segments

In [6]:
'''
这个函数通过调用 `get_next_sentence` 函数从输入的段落生成用于下一句预测的训练样本。

'''
def get_nsp_data_from_paragraph(
    paragraph, paragraphs, vocab, max_len
):
    '''
    **初始化数据列表**
    nsp_data_from_paragraph 用于存储生成的训练样本。
    '''
    nsp_data_from_paragraph = []
    '''
    **遍历段落中的句子**
    遍历段落中的句子，对每一对相邻句子调用 `get_next_sentence` 函数生成训练样本。
    `paragraph[i]` 和 `paragraph[i + 1]` 是相邻的两个句子。
    '''
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = get_next_sentence(
            paragraph[i], paragraph[i+1], paragraphs
        )
        '''
        **检查长度约束**
        
        考虑1个'<cls>'词元和2个'<sep>'词元
        
        检查生成的句子对的长度是否超过 `max_len`（考虑到一个 `<cls>` 词元和两个 `<sep>` 词元）。
        如果超过，则跳过这个样本。
        '''
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        '''
        **获取词元和片段**
        使用 `get_tokens_and_segments` 函数获取词元和片段标记。
        '''
        # tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        tokens, segments = get_tokens_and_segments(tokens_a, tokens_b)
        
        '''
        **添加到数据列表**
        
        将生成的词元、片段标记和 `is_next` 标志
        添加到 `nsp_data_from_paragraph` 列表中。
        '''
        nsp_data_from_paragraph.append(
            (tokens, segments, is_next)
        )
        
    return nsp_data_from_paragraph

总结

这段代码的两个辅助函数 `_get_next_sentence` 和 `_get_nsp_data_from_paragraph` 分别实现了生成下一句预测任务的单个样本和从段落生成多个训练样本的功能。通过这些函数，可以将原始的文本语料库转换为适合BERT预训练的数据格式。

---

`replace_mlm_tokens`,`get_mlm_data_from_tokens`'的目的是:
生成用于BERT模型预训练的遮蔽语言模型（MLM）任务的数据。
MLM任务通过随机遮蔽输入序列中的一些词元，并让模型预测这些被遮蔽的词元，从而使模型能够更好地理解上下文。

---

生成遮蔽语言模型任务的数据

In [7]:
'''
这个函数负责将输入序列中的一部分词元替换为 `<mask>` 或其他随机词元，
并记录这些替换的位置和原始词元。

'''
def replace_mlm_tokens(
    tokens, 
    candidate_pred_positions, 
    num_mlm_preds,
    vocab
):
    '''
    为遮蔽语言模型的输入创建新的词元副本，
    其中输入可能包含替换的“<mask>”或随机词元
    
    创建一个新的词元列表 `mlm_input_tokens`，
    它是输入 `tokens` 的副本，用于进行遮蔽和替换操作。
    '''
    mlm_input_tokens = [token for token in tokens]
    
    '''
    初始化一个空列表 `pred_positions_and_labels`，
    用于存储被替换的词元位置及其原始词元。
    '''
    pred_positions_and_labels = []
    
    '''
    打乱后用于在遮蔽语言模型任务中获取15%的随机词元进行预测
    
    将候选的预测位置 `candidate_pred_positions` 随机打乱，
    以便从中随机选择15%的词元进行预测。
    '''
    random.shuffle(candidate_pred_positions)
    
    '''
    **替换词元**
    
    对于每一个候选的预测位置 `mlm_pred_position`，根据以下概率进行替换：
    - 80%的概率将词元替换为 `<mask>`。
    - 10%的概率保持词元不变。
    - 10%的概率将词元替换为词汇表中的随机词元。

    每次替换后，将位置和原始词元添加到 `pred_positions_and_labels` 中，
    并更新 `mlm_input_tokens`。

    '''
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        '''
        80%的时间：将词替换为“<mask>”词元
        '''
        masked_token = None
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            '''
            10%的时间：保持词不变
            '''
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            else:
                '''
                10%的时间：用随机词替换该词
                '''
                masked_token = random.choice(vocab.idx_to_token)
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append(
            (
                mlm_pred_position, tokens[mlm_pred_position]
            )
        )
        '''
        返回可能替换后的输入词元列表 `mlm_input_tokens`，
        以及发生预测的词元位置和原始词元的列表 `pred_positions_and_labels`
        '''
        return mlm_input_tokens, pred_positions_and_labels

In [8]:
'''
这个函数通过调用 `_replace_mlm_tokens` 函数，生成用于遮蔽语言模型任务的数据。
'''
def get_mlm_data_from_tokens(tokens, vocab):
    '''
    **初始化候选位置列表**
    
    遍历输入的词元列表 `tokens`，
    将所有非特殊词元的位置添加到 `candidate_pred_positions` 列表中。
    
    特殊词元（如 `<cls>` 和 `<sep>`）不会在遮蔽语言模型任务中被预测。
    '''
    candidate_pred_positions = []
    '''
    tokens是一个字符串列表
    '''
    for i, token in enumerate(tokens):
        '''
        在遮蔽语言模型任务中不会预测特殊词元
        '''
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    '''
    **计算要预测的词元数量**
    计算需要预测的词元数量，约为总词元数量的15%。确保至少预测一个词元。
    
    遮蔽语言模型任务中预测15%的随机词元
    '''
    num_mlm_preds = max(
        1, round(len(tokens) * 0.15)
    )
    '''
    **调用替换函数**
    调用 `_replace_mlm_tokens` 函数，生成遮蔽后的输入词元和预测位置及其原始词元。
    '''
    mlm_input_tokens, pred_positions_and_labels = replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab
    )
    '''
    **排序预测位置和标签**
    
    对预测位置和标签进行排序，
    并分别提取预测位置 `pred_positions` 和预测标签 `mlm_pred_labels`。
    '''
    pred_positions_and_labels = sorted(
        pred_positions_and_labels, key=lambda x: x[0]
    )
    
    pred_positions = [
        v[0] for v in pred_positions_and_labels
    ]
    mlm_pred_labels = [
        v[1] for v in pred_positions_and_labels
    ]
    '''
    返回词汇表中对应的输入词元索引 `vocab[mlm_input_tokens]`，
    预测位置 `pred_positions`，以及预测标签的索引 `vocab[mlm_pred_labels]`。
    '''
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]


总结

这段代码定义了两个函数 `_replace_mlm_tokens` 和 `_get_mlm_data_from_tokens`，用于生成BERT预训练任务中的遮蔽语言模型数据。前者负责将部分词元替换为 `<mask>` 或随机词元，并记录替换位置和原始词元；后者负责调用前者，生成遮蔽后的输入词元及其对应的预测位置和标签。通过这些函数，可以将原始的文本序列转换为适合BERT预训练的数据格式。

---

将文本转化为预训练数据集

pad_bert_inputs, WikiTextDataset, load_data_wiki的目的是:
    将文本数据转换为适合BERT模型预训练的数据集，包括下一句预测（NSP）和遮蔽语言模型（MLM）任务。代码实现了数据的预处理、词元化、填充以及数据加载。

In [9]:
'''
这个函数用于将BERT的输入填充到统一的长度，并准备好所有训练所需的数据。
'''

def pad_bert_inputs(
    examples, max_len, vocab
):
    '''
    初始化多个列表，用于存储填充后的各项数据。
    `max_num_mlm_preds` 表示遮蔽语言模型任务中最多的预测词元数（即输入序列长度的15%）。
    '''
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = [] 
    '''
    **遍历每个示例**：
    遍历输入的每个示例，每个示例包含了BERT输入序列的各项信息。
    '''
    for (
        token_ids, pred_positions, mlm_pred_label_ids, segments, is_next
    ) in examples:
        '''
        **填充和处理各项数据**：

        - `token_ids` 填充到 `max_len` 长度，填充的词元为 `<pad>`。
        '''
        all_token_ids.append(
            torch.tensor(
                token_ids + [vocab['<pad>']] * (max_len - len(token_ids)),
                dtype=torch.long
            )
        )
        '''
        - `segments` 填充到 `max_len` 长度，填充的值为 0。
        '''
        all_segments.append(
            torch.tensor(
                segments + [0] * (max_len - len(token_ids)),
                dtype=torch.long
            )
        )
        '''
        - `valid_lens` 表示真实的词元长度，不包括填充部分。
        '''
        valid_lens.append(
            torch.tensor(
                len(token_ids),
                dtype=torch.float32
            )
        )
        '''
        - `pred_positions` 填充到 `max_num_mlm_preds` 长度，填充的值为 0。
        '''
        all_pred_positions.append(
            torch.tensor(
                pred_positions + [0]*(max_num_mlm_preds - len(pred_positions)),
                dtype=torch.long
            )
        )
        '''
        - `mlm_weights` 用于遮蔽语言模型的损失计算，填充值为0的部分不会在损失中计算。
        '''
        all_mlm_weights.append(
            torch.tensor(
                [1.0] * len(mlm_pred_label_ids) + [0.0] * (max_num_mlm_preds - len(pred_positions)),
                dtype=torch.float32
            )
            
        )
        '''
        - `mlm_labels` 填充到 `max_num_mlm_preds` 长度，填充的值为 0。
        '''
        all_mlm_labels.append(
            torch.tensor(
                mlm_pred_label_ids + [0] * (max_num_mlm_preds - len(mlm_pred_label_ids )),
                dtype=torch.long
            )
        )
        '''
        - `nsp_labels` 存储下一句预测的标签。
        '''
        nsp_labels.append(
            torch.tensor(
                is_next, dtype=torch.long
            )
        )
    '''
    返回填充后的各项数据。
    '''
    return (
        all_token_ids,
        all_segments,
        valid_lens,
        all_pred_positions,
        all_mlm_weights,
        all_mlm_labels,
        nsp_labels
    )
        

In [10]:
'''
这个类用于将WikiText-2数据集转换为适合BERT预训练的数据格式。
'''
class WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        '''
        - 将段落 `paragraphs` 进行词元化，结果是一个列表的列表，每个列表是一个段落的词元列表。
        '''
        paragraphs = [
            d2l.tokenize(
                paragraph,
                token='word'
            ) for paragraph in paragraphs
        ]
        '''
        - 将所有段落中的句子提取出来，形成 `sentences` 列表。
        '''
        sentences = [
            sentence for paragraph in paragraphs for sentence in paragraph
        ]
        '''
        - 创建词汇表 `vocab`，过滤掉出现次数少于5次的词元，并预留特殊词元。
        '''
        self.vocab = d2l.Vocab(
            sentences, 
            min_freq=5, 
            reserved_tokens=[
                '<pad>', '<mask>', '<cls>', '<sep>'
            ]
        )
        '''
        **生成NSP任务的数据**
        调用 `_get_nsp_data_from_paragraph` 函数，
        为每个段落生成下一句预测任务的数据。
        '''
        examples = []
        for paragraph in paragraphs:
            examples.extend(
                get_nsp_data_from_paragraph(
                    paragraph, paragraphs, self.vocab, max_len
                )
            )
        '''
        **生成MLM任务的数据**：
        调用 `get_mlm_data_from_tokens` 函数，
        为每个示例生成遮蔽语言模型任务的数据。
        '''
        examples = [
            (
                get_mlm_data_from_tokens(
                    tokens, self.vocab
                ) + (
                    segments, is_next
                )
            ) for tokens, segments, is_next in examples
        ]
        '''
        **填充数据**：
        调用 `_pad_bert_inputs` 函数，将所有示例的数据进行填充。  
        '''
        (
            self.all_token_ids,
            self.all_segments,
            self.valid_lens,
            self.all_pred_positions,
            self.all_mlm_weights,
            self.all_mlm_labels,
            self.nsp_labels
        ) = pad_bert_inputs(
            examples, max_len, self.vocab
        )
    '''
    - `__getitem__` 方法用于获取指定索引 `idx` 的示例数据。
    '''
    def __getitem__(self, idx):
        return (
            self.all_token_ids[idx],
            self.all_segments[idx],
            self.valid_lens[idx],
            self.all_pred_positions[idx],
            self.all_mlm_weights[idx],
            self.all_mlm_labels[idx],
            self.nsp_labels[idx]
        )
    '''
    - `__len__` 方法返回数据集的大小。
    '''
    def __len__(self):
        return len(self.all_token_ids)

In [11]:
'''
这个函数用于加载WikiText-2数据集并生成用于预训练的样本。
'''
def load_data_wiki(batch_size, max_len):
    num_workers = d2l.get_dataloader_workers()
    data_dir = './data/wikitext-2/'
    paragraphs = read_wiki(data_dir)
    train_set = WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(
        train_set, 
        batch_size, 
        shuffle=True,
        num_workers=num_workers
    )
    return train_iter, train_set.vocab

In [12]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

for (
    tokens_X, 
    segments_X,
    valid_lens_X,
    pred_positions_X,
    mlm_weights_X,
    mlm_Y,
    nsp_Y
) in train_iter:
    print(
        tokens_X.shape,
        segments_X.shape,
        valid_lens_X.shape,
        pred_positions_X.shape,
        mlm_weights_X.shape,
        mlm_Y.shape,
        nsp_Y.shape
    )
    break

torch.Size([512, 64]) torch.Size([512, 64]) torch.Size([512]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512])


In [13]:
len(vocab)

20256